In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-21 16:34:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  3.52MB/s    in 0.3s    

2022-04-21 16:34:27 (3.52 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [11]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   44552694|             7|
|   27457155|             6|
|   35842367|             9|
|   16848166|             5|
|   13313689|            10|
|    3628854|             1|
|   43626894|             1|
|    4658983|             8|
|   16457323|             3|
|   30024726|             5|
|    3636174|             1|
|     582750|             1|
|      78113|             1|
|    2246607|             5|
|   26264517|             1|
|    7439900|             1|
|    2361059|             2|
|   22965400|             2|
|   32777038|             1|
|   29019526|             2|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004GVKBTO|           Blackjack|
|B004HE63SI|Chopin Classical ...|
|B004N0XPLY|       Call on Faith|
|B004O4QYVM|Learn To Play The...|
|B004OAPM9Q|              Roster|
+----------+--------------------+
only showing top 5 rows



In [15]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3PMQXEVYR4ZT1|    1774101|B00DVKOYBM|     263046246| 2015-08-31|
|R1I1K3EO98EMUV|   13679234|B00R1IA4RS|     128868854| 2015-08-31|
|R3K23URSI2MY42|   15056632|B01069LGHK|     955481764| 2015-08-31|
| RH2P770T10X38|   52477711|B00X64ZKM0|     455029830| 2015-08-31|
| RV9YBAW8CWFYN|   15021042|B00KIZO238|     685358159| 2015-08-31|
| R19627K7G1G6X|    5181462|B00H9FE0FU|     158085302| 2015-08-31|
|R22858XECEF7YQ|   19360265|B0064X7FVE|     140639925| 2015-08-31|
|R3KQOY7MDZOK0U|   28928572|B00N28818A|     838378762| 2015-08-31|
|R31BKIN02QFDHS|   26842555|B00HGMXSGI|     141073643| 2015-08-31|
|R3FW62P5EUDUUV|   25535583|B0119R2DB6|     498391460| 2015-08-31|
| RBD8ROQJQO68E|    3343825|B00O0OR65E|     156086007| 2015-08-31|
|R1S04R7IT12FKH|   49181739|B00YCL9R0W|     629058532| 2015-08

In [16]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PMQXEVYR4ZT1|          3|            0|          0|   N|                Y|
|R1I1K3EO98EMUV|          5|            0|          0|   N|                Y|
|R3K23URSI2MY42|          5|            4|          4|   N|                Y|
| RH2P770T10X38|          5|            0|          0|   N|                Y|
| RV9YBAW8CWFYN|          1|            0|          0|   N|                Y|
| R19627K7G1G6X|          5|            0|          0|   N|                Y|
|R22858XECEF7YQ|          3|            0|          0|   N|                N|
|R3KQOY7MDZOK0U|          1|            0|          0|   N|                Y|
|R31BKIN02QFDHS|          4|            0|          0|   N|                Y|
|R3FW62P5EUDUUV|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [19]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<db>"
config = {"user":"postgres", 
          "password": "<password", 
          "driver":"org.postgresql.Driver"}

In [20]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [22]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [23]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)